In [1]:
from xyz2graph import MolGraph, to_networkx_graph, to_plotly_figure
from plotly.offline import init_notebook_mode, iplot
import numpy as np
import pickle
from rdkit import Chem 
from pathlib import Path

RDKit WARNING: [20:01:33] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
def mapping_xyz(mg, adj_atoms):
    x = []
    y = []
    z = []
    for i, j in adj_atoms:
        x += (mg.x[i], mg.x[j], None)
        y += (mg.y[i], mg.y[j], None)
        z += (mg.z[i], mg.z[j], None)
    x = np.array(x).reshape(-1, 3)
    y = np.array(y).reshape(-1, 3)
    z = np.array(z).reshape(-1, 3)
    return x, y, z



def show_one_line(trace, points, selector):
    if points.point_inds:
        idx = points.point_inds[0]              
        mask = np.array([False] * len(adj_atoms))
        for i, pair  in enumerate(adj_atoms):
            if idx in pair:
                mask[i] = True

        new_x = x[mask, :].reshape(-1).tolist()
        new_y = y[mask, :].reshape(-1).tolist()
        new_z = z[mask, :].reshape(-1).tolist()


        if idx in mg.mapping_list:
            with f.batch_update(): 
                mapping_scatter['x'] = new_x
                mapping_scatter['y'] = new_y
                mapping_scatter['z'] = new_z

def show_no_line(trace, points, selector):
    print('leave')
    with f.batch_update():
        mapping_scatter['x'] = []
        mapping_scatter['y'] = []
        mapping_scatter['z'] = []


In [29]:
item = '/root/repo/prepare/4G3E_2/SYSTEM/Protein/inter2~mol409/morph.gro'
mg = MolGraph()
mg.read_grofile(item)
f = to_plotly_figure(mg)
iplot(f)

In [28]:
file_name = '/root/repo/prepare/4G3E_2/SYSTEM/Protein/mol418~inter1/morph.gro'
fp = Path(file_name).parent
pert_name = fp.name
root = fp.parent.parent
p1 = pert_name.split('~')[0] + '_aligned_' + pert_name 
p2 = pert_name.split('~')[1] + '_aligned_' + pert_name


item=root/Path(p1)
# item=root/Path(p2)
# item = str(Path(file_name).with_name(p2))
pick = str(item) + '.pickle'
mg = MolGraph()

with open(pick, 'rb') as f:
    bundle = pickle.load(f)

mol1 = bundle['mol1']
mol2 = bundle['mol2']
mapping = bundle['mcs']

mg.draw_mol(mol1)
mg.draw_mol(mol2, pos_offset=10)
mg.generate_mapping(mapping)


# Create the Plotly figure object
f = to_plotly_figure(mg)
# f.layout.hovermode = 'closest'

adj_atoms = []
for atom, neighbors in mg.mapping_list.items():
    for neigh in neighbors:
        adj_atoms.append([atom, neigh])



atom_scatter, bond_scatter, mapping_scatter = f.data

x, y, z = mapping_xyz(mg, adj_atoms)

atom_scatter.on_hover(show_one_line)                
# atom_scatter.on_unhover(show_no_line)
f

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'marker': {'color': [black, black, black, blac…

In [22]:
p1

'inter1_aligned_inter1~mol419'